In [2]:
import numpy as np
from scipy.sparse import tril, csr_matrix
from scipy.io import mmread

def ichol_sparse_optimized(A, drop_tol=1e-5):
    """
    Optimized Incomplete Cholesky factorization for a sparse matrix A.
    A should be a symmetric positive definite matrix in CSC format.
    """
    A = A.tocsc()  # Ensure A is in CSC format
    n = A.shape[0]
    L = A.copy()   # Create a copy of A to store the result

    for k in range(n):
        # Update the diagonal element
        diag = L[k, k]
        if diag <= 0:
            raise ValueError(f"Matrix is not positive definite at row {k}.")
        L[k, k] = np.sqrt(diag)

        # Update column k below the diagonal
        start, end = L.indptr[k], L.indptr[k + 1]
        rows = L.indices[start:end]
        below_diag_mask = rows > k

        if np.any(below_diag_mask):
            rows = rows[below_diag_mask]
            L.data[start:end][below_diag_mask] /= L[k, k]

            # Update the lower-right submatrix
            for i in rows:
                i_start, i_end = L.indptr[i], L.indptr[i + 1]
                row_indices = L.indices[i_start:i_end]
                overlap_mask = np.isin(row_indices, rows, assume_unique=True)
                overlap = row_indices[overlap_mask]

                # Drop values below the tolerance
                for j in overlap:
                    L[i, j] -= L[i, k] * L[j, k]
                    if abs(L[i, j]) < drop_tol:
                        L[i, j] = 0

    # Extract the lower triangular part of L
    L = tril(L, format='csr')
    return L

# Load the matrix
file_path = 'gyro_k.mtx'  # Update this with your actual file path
A = mmread(file_path)

# Perform the optimized Incomplete Cholesky factorization
try:
    L = ichol_sparse_optimized(A)
    print("Incomplete Cholesky factorization successful.")
except Exception as e:
    print(f"Error during factorization: {e}")


Error during factorization: Matrix is not positive definite at row 4682.
